# Data Acquisition

In [4]:
from datetime import datetime
import time
import opendota
import pandas as pd
import numpy as np
import json
import os

In [5]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [135]:
def get_match_list(datetime_start_unix, dateime_end_unix):
  '''Queries the OpenDota "public_matches" table between two datetimes using PostgreSQL and saves a csv called matches_yyyymmdd_hhmmss.csv'''
  client = opendota.OpenDota()

  result_size = 500000
  start_time = datetime_start_unix
  i=0

  # Divides the requests into 500000 chunks, due to size limitations when querying the OpenDota API
  while result_size == 500000:
    i=i+1
    output_cur = client.explorer(f'SELECT * FROM public_matches where start_time > {start_time} AND start_time < {dateime_end_unix} LIMIT 500000')
    start_time = output_cur[-1]['start_time']
    result_size = len(output_cur)
    df_output_cur = pd.DataFrame(output_cur)
    
    if i == 1:
      df_output = df_output_cur
    else:
      df_output = pd.concat([df_output, df_output_cur], axis=0)

    print('Loop ' + str(i) +' - Size ' + str(len(df_output))) 
  
  return df_output

In [7]:
# Generate match list
# Dates for fixed hero time:
# Marci added 28/10/2021, so 29/10/2021 = 1635462000
# Primal Beast added 23/02/2022, so 22/02/2022 = 1645488000
# Above time window is ~4 months

# Dates for Primal Beast added
# Primal Beast added 23/02/2022, so 24/02/2022 = 1645660800
# Up to 2 months after, so 24/04/2022 = 1650758400

### Sometimes the first few runs do not work due to no response for the OpenDota API - keep trying and it will eventually work
df_match_list = get_match_list(1635462000,1645488000)
now = datetime.now()
df_match_list.to_csv(f'match_list_{now.year:04d}{now.month:02d}{now.day:02d}_{now.hour:02d}{now.minute:02d}{now.second:02d}.csv', index=False)

Loop 1 - Size 500000
Loop 2 - Size 1000000
Loop 3 - Size 1500000
Loop 4 - Size 2000000
Loop 5 - Size 2500000
Loop 6 - Size 3000000
Loop 7 - Size 3500000
Loop 8 - Size 4000000
Loop 9 - Size 4500000
Loop 10 - Size 5000000
Loop 11 - Size 5500000
Loop 12 - Size 5600752


In [37]:
def get_match_picks(match_ids):
    query_size = 1000 # allows multiple match_ids per query, reducing the total number of queries and improving extraction speed (OpenDota only returns around 1000 at a time)
    client = opendota.OpenDota()
    columns = ['match_id'] + [f'hero{i}_pick' for i in range(0,10)] + [f'hero{i}_slot' for i in range(0,10)]
    num_matches = len(match_ids)
    df_picks = pd.DataFrame(
        data=np.zeros([num_matches,len(columns)]),
        columns=columns)

    count = 0
    row = -1

    match_ids_current = []
    for match_id in match_ids:
        count+=1
        match_ids_current.append(str(match_id)) # builds up a list of match_ids until reaching query size, or until last match reached
        if (count==query_size) | ((num_matches-row-len(match_ids_current))==1):
            count = 0 # reset count
            match_ids_current_str = ','.join(match_ids_current) # expands list in comma-separated string for use in PostgreSQL query
            query = f'SELECT * FROM public_player_matches WHERE match_id IN ({match_ids_current_str})'
            picks = client.explorer(query) # query OpenDota
            
            # loop through results (each item is a match id pick, there *should* be 10 picks per match id)
            match_id_cur = -1 # memory of current match_id_cur, allows moving to next row in output df
            hero = -1 # initial hero
            for pick in picks:
                # new match_id means move to next row
                if pick['match_id'] != match_id_cur:
                    match_id_cur = int(pick['match_id'])
                    row+=1 # next row for next match
                    hero=-1 # reset first hero
                    
                    df_picks.loc[df_picks.index[row], 'match_id'] = match_id_cur # assign match_id column
                
                hero+=1
                df_picks.loc[df_picks.index[row], f'hero{hero}_pick'] = pick['hero_id']
                df_picks.loc[df_picks.index[row], f'hero{hero}_slot'] = pick['player_slot']

            match_ids_current = [] # reset current match_id batch
            print(f'{row+1} matches picks extracted')
    
    return df_picks

In [7]:
matches = pd.read_csv('matches_2.csv')

In [38]:
df_picks = get_match_picks(matches['match_id'][0:3500].values)
df_picks.to_csv('picks_2.csv', index=False)

1000 matches picks extracted
2000 matches picks extracted
3000 matches picks extracted
3500 matches picks extracted


In [33]:
# Query debugger
# out = client.explorer("SELECT * FROM public_player_matches WHERE match_id IN (6246229802,6246229803,6246229805,6246229904,6246229807,6246230009,6246229915,6246229809,6246229905,6246229907)")
client = opendota.OpenDota() #6447015200
out = client.explorer("SELECT * FROM public_player_matches WHERE match_id IN (6246229802)")
print(out)

[]


In [5]:
def download_match_heroes(match_ids):
    '''Uses the PyOpenDota function "get_match" to return match information about each "match_id" in iterable argument "matches"
    Saves heroes picked/banned for each match_id in a json file called match-id.json'''
    client = opendota.OpenDota()
    total = len(match_ids)
    count = 0
    for match_id in match_ids:
        count += 1
        print(str(count) + '/' + str(total))
        try:
            match = client.get_match(match_id)
            matchjson = json.dumps(match)
            f = open(f'match_jsons\\{match_id}.json','w')
            f.write(matchjson)
            f.close()
        except:
            pass


In [ ]:
# Fetch selected heroes for each match_id
matches = pd.read_csv('match_list_.csv') # replace argument with match_list .csv file
download_match_heroes(matches['match_id'])

In [6]:
def check_hero_picks(picks_bans):
    '''Function takes a list of pick+ban dictionaries and returns boolean with whether there were a total of 10 picks and 5 on each team
    True: 10 total picks, 5 team0 picks, 5 team1 picks, picks were in first ten records in list
    False: otherwise'''

    picks = [np.nan]*10
    valid = False
    picks_total = 0
    picks_team0 = 0
    picks_team1 = 0
    count=0
    first_ten = True
    for pickban in picks_bans:
        if pickban['is_pick']==True:
            picks_total+=1
            if count>9:
                first_ten = False
            if pickban['team']==0:
                if picks_team0<5:
                    picks[picks_team0] = pickban['hero_id']
                picks_team0+=1
            elif pickban['team']==1:
                if picks_team1<5:
                    picks[picks_team1+5] = pickban['hero_id']
                picks_team1+=1
        count+=1

    if (picks_total==10) & (picks_team0==5) & (picks_team1==5) & first_ten:
        valid = True
    
    output = {'valid': valid, 'picks': picks}

    return output

In [19]:
def compile_match_heroes():
    matches = pd.read_csv('match_list_.csv') # replace argument with match list csv
    num_matches = len(matches)
    dir_match_jsons = 'match_jsons/'
    # Results df
    df_heroes = pd.DataFrame(columns=['match_id','hero0','hero1','hero2','hero3','hero4','hero5','hero6','hero7','hero8','hero9','heroes_valid'])

    count = 0
    for idx, match in matches.iterrows():
        match_id = match['match_id']
        if match_id>6447015601:
            break
        # Try see if json for match exists
        try:
            picks_bans = json.load(open(f'{dir_match_jsons}{match_id}.json'))['picks_bans']
            picksbans_checked = check_hero_picks(picks_bans)
            df_heroes.loc[count,:] = [match_id] + picksbans_checked['picks'] + [picksbans_checked['valid']]
            
        # If no match json, include row of nans
        except:
            # print('exception')
            df_heroes.loc[count,:] = [match_id] + [np.nan]*10 + [False]
        count+=1
        print(f'{count}/{num_matches} - {match_id}')

    now = datetime.now()        
    df_heroes.to_csv(f'match_heroes_{now.year:04d}{now.month:02d}{now.day:02d}_{now.hour:02d}{now.minute:02d}{now.second:02d}.csv', index=False)

In [20]:
# Compile jsons into updated "matches" table
compile_match_heroes()

1/5600752 - 6447015200
2/5600752 - 6447015219
3/5600752 - 6447015314
4/5600752 - 6447015315
5/5600752 - 6447015300
6/5600752 - 6447015512
7/5600752 - 6447015301
8/5600752 - 6447015319
9/5600752 - 6447015304
10/5600752 - 6447015302
11/5600752 - 6447015410
12/5600752 - 6447015411
13/5600752 - 6447015514


In [23]:
# Merge matches and heroes datasets
df_match_list = pd.read_csv('match_list_.csv') # replace argument with match list csv
df_match_heroes = pd.read_csv('match_heroes_.csv') # replace argument with match hero csv
df_match_merged = df_match_list.merge(df_match_heroes, on='match_id', how='left')

# Save merged
now = datetime.now()
df_match_merged.to_csv(f'match_merged_{now.year:04d}{now.month:02d}{now.day:02d}_{now.hour:02d}{now.minute:02d}{now.second:02d}.csv', index=False)

In [2]:
client = opendota.OpenDota()

In [22]:
client.get_schema('public_player_matches')

{'match_id': 'bigint', 'player_slot': 'integer', 'hero_id': 'integer'}

In [24]:
client.explorer("SELECT * FROM public_player_matches WHERE match_id=6447015219 LIMIT 10")

[{'match_id': 6447015219, 'player_slot': 0, 'hero_id': 137},
 {'match_id': 6447015219, 'player_slot': 1, 'hero_id': 136},
 {'match_id': 6447015219, 'player_slot': 2, 'hero_id': 63},
 {'match_id': 6447015219, 'player_slot': 3, 'hero_id': 75},
 {'match_id': 6447015219, 'player_slot': 4, 'hero_id': 12},
 {'match_id': 6447015219, 'player_slot': 128, 'hero_id': 36},
 {'match_id': 6447015219, 'player_slot': 129, 'hero_id': 94},
 {'match_id': 6447015219, 'player_slot': 130, 'hero_id': 76},
 {'match_id': 6447015219, 'player_slot': 131, 'hero_id': 14},
 {'match_id': 6447015219, 'player_slot': 132, 'hero_id': 5}]

In [38]:
out = client.explorer("SELECT * FROM public_player_matches WHERE match_id IN (6538637102)")
print(out)

[{'match_id': 6538637102, 'player_slot': 0, 'hero_id': 94}, {'match_id': 6538637102, 'player_slot': 1, 'hero_id': 23}, {'match_id': 6538637102, 'player_slot': 2, 'hero_id': 18}, {'match_id': 6538637102, 'player_slot': 3, 'hero_id': 42}, {'match_id': 6538637102, 'player_slot': 4, 'hero_id': 48}, {'match_id': 6538637102, 'player_slot': 128, 'hero_id': 75}, {'match_id': 6538637102, 'player_slot': 129, 'hero_id': 97}, {'match_id': 6538637102, 'player_slot': 130, 'hero_id': 74}, {'match_id': 6538637102, 'player_slot': 131, 'hero_id': 2}, {'match_id': 6538637102, 'player_slot': 132, 'hero_id': 1}]


In [178]:
# out = client.explorer("SELECT * FROM public_player_matches WHERE match_id IN (6246229802,6246229803,6246229805,6246229904,6246229807,6246230009,6246229915,6246229809,6246229905,6246229907)")
client = opendota.OpenDota()
out = client.explorer("SELECT * FROM public_player_matches WHERE match_id IN (6246229802)")
print(out)

[]


In [80]:
ids = list(pd.read_csv('ids.csv')['match_id'].values)
ids = [str(id) for id in ids]

In [91]:
ids_sub = ','.join(ids[0:1000])
out = client.explorer(f"SELECT * FROM public_player_matches WHERE match_id IN ({ids_sub})")
len(out)

10000

In [92]:
out

[{'match_id': 6447015200, 'player_slot': 0, 'hero_id': 94},
 {'match_id': 6447015200, 'player_slot': 1, 'hero_id': 87},
 {'match_id': 6447015200, 'player_slot': 2, 'hero_id': 40},
 {'match_id': 6447015200, 'player_slot': 3, 'hero_id': 71},
 {'match_id': 6447015200, 'player_slot': 4, 'hero_id': 22},
 {'match_id': 6447015200, 'player_slot': 128, 'hero_id': 128},
 {'match_id': 6447015200, 'player_slot': 129, 'hero_id': 48},
 {'match_id': 6447015200, 'player_slot': 130, 'hero_id': 57},
 {'match_id': 6447015200, 'player_slot': 131, 'hero_id': 4},
 {'match_id': 6447015200, 'player_slot': 132, 'hero_id': 137},
 {'match_id': 6447015219, 'player_slot': 0, 'hero_id': 137},
 {'match_id': 6447015219, 'player_slot': 1, 'hero_id': 136},
 {'match_id': 6447015219, 'player_slot': 2, 'hero_id': 63},
 {'match_id': 6447015219, 'player_slot': 3, 'hero_id': 75},
 {'match_id': 6447015219, 'player_slot': 4, 'hero_id': 12},
 {'match_id': 6447015219, 'player_slot': 128, 'hero_id': 36},
 {'match_id': 6447015219,

In [39]:
out[0]['match_id']

6538637102

In [16]:
client.explorer("SELECT * FROM api_keys LIMIT 1")

[]

In [34]:
df_match_list = pd.read_csv('match_list_20220723_135613.csv')